# Notebook Summary
In this notebook I test various decision tree based models

In [2]:
import autoreload
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import Classes

In [3]:
import pickle
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score, recall_score, precision_recall_curve,f1_score, fbeta_score
from imblearn.over_sampling import ADASYN
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

# Pickle in Data

In [4]:
# Pickle in cleaned dataframe

# Designate path

path = r"C:\Users\Andrew\Documents\Metis\TikTok_Song_Predictor\Pickle\df_agg.pkl"

df = pickle.load(open(path,'rb'))
df.head(2)

,level_0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,TikTok Link,Release Date,Position Change,spotify_uri,audio_analysis,feature_analysis,success,year,top_albums,top_artists
0,0,0.880,0.501,2.0,-6.774,1.0,0.062,0.0494,0.0695,0.4360,...,https://www.tiktok.com/music/All-TikTok-Mashup...,2020-08-17,23.0,5TpvLkESnw1g9wDz52efeO,"{'meta': {'analyzer_version': '4.0.0', 'platfo...","{'danceability': 0.88, 'energy': 0.501, 'key':...",1,2020.0,Other,Other
1,162,0.935,0.454,1.0,-7.509,1.0,0.375,0.0194,0.0000,0.0824,...,https://www.tiktok.com/music/WAP-Megan-Thee-St...,2018-03-22,15.0,4Oun2ylbjFKMPTiaSbbCih,"{'meta': {'analyzer_version': '4.0.0', 'platfo...","{'danceability': 0.935, 'energy': 0.454, 'key'...",1,2018.0,Other,Cardi B


# 1) Random Forest

In [5]:
# Seperate features from label

X = df.loc[:,['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo','year']]

y = df['success']

In [6]:
#Split data into 3: 60% train, 20% validation, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25,stratify=y_train)

In [7]:
# Oversample training data
ada = ADASYN(random_state=42)
X_adasyn_tr, y_adasyn_tr = ada.fit_resample(X_train,y_train)

In [8]:
# run random forest
clf = RandomForestClassifier()
clf.fit(X_adasyn_tr, y_adasyn_tr)
y_predict = clf.predict(X_val)

#scores
print("Scores for the clf")
print("Training score: {:6.2f}%".format(100*clf.score(X_adasyn_tr, y_adasyn_tr)))
print("Val set score: {:6.2f}%".format(100*clf.score(X_val, y_val)))

#precision/recall
print("\nPrecision / Recall")
print("Val F1 score: {:6.2f}%".format(f1_score(clf.predict(X_val), y_val)))
print("Precision: {:6.2f}%,   Recall: {:6.2f}%".format(100*precision_score(y_val, y_predict), 
                                                     100*recall_score(y_val, y_predict)))

Scores for the clf
Training score: 100.00%
Val set score:  83.13%

Precision / Recall
Val F1 score:   0.25%
Precision:  28.03%,   Recall:  23.16%


# 2) Balanced Random Forest Classifier

In [9]:
# Seperate features from label

X = df.loc[:,['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo','year']]

y = df['success']

In [10]:
#Split data into 3: 60% train, 20% validation, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25,stratify=y_train)

In [11]:
model = BalancedRandomForestClassifier(n_estimators=10)

In [12]:
# run model
model.fit(X_train, y_train)
y_predict = model.predict(X_val)

#scores
print("Scores for the random forest")
print("Training score: {:6.2f}%".format(100*model.score(X_train, y_train)))
print("Test set score: {:6.2f}%".format(100*model.score(X_val, y_val)))

#precision/recall
print("\nPrecision / Recall")
print("Val F1 score: {:6.2f}%".format(100*f1_score(y_predict, y_val)))
print("Val F2 score: {:6.2f}%".format(100*fbeta_score(y_predict, y_val,beta=0.5)))
print("Precision: {:6.2f}%".format(100*precision_score(y_val, y_predict)))
print("Recall: {:6.2f}%".format(100*recall_score(y_val, y_predict)))

Scores for the random forest
Training score:  75.24%
Test set score:  66.78%

Precision / Recall
Val F1 score:  34.11%
Val F2 score:  49.11%
Precision:  22.60%
Recall:  69.47%


This is the best model - let's pickle it for outside data

In [13]:
# Pickle model
path = r"C:\Users\Andrew\Documents\Metis\TikTok_Song_Predictor\Pickle\brfc_model.pkl"
pickle.dump(model, open(path, 'wb'))

# 3) Balanced Bagging Classifier

In [12]:
# Seperate features from label

X = df.loc[:,['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo','year']]

y = df['success']

In [13]:
#Split data into 3: 60% train, 20% validation, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25,stratify=y_train)

In [14]:
# define model
model = BalancedBaggingClassifier()

In [15]:
# run model
model.fit(X_train, y_train)
y_predict = model.predict(X_val)

#scores
print("Scores for the random forest")
print("Training score: {:6.2f}%".format(100*model.score(X_train, y_train)))
print("Test set score: {:6.2f}%".format(100*model.score(X_val, y_val)))

#precision/recall
print("\nPrecision / Recall")
print("Val F1 score: {:6.2f}%".format(100*f1_score(y_predict, y_val)))
print("Val F2 score: {:6.2f}%".format(100*fbeta_score(y_predict, y_val,beta=0.5)))
print("Precision: {:6.2f}%".format(100*precision_score(y_val, y_predict)))
print("Recall: {:6.2f}%".format(100*recall_score(y_val, y_predict)))

Scores for the random forest
Training score:  84.58%
Test set score:  72.05%

Precision / Recall
Val F1 score:  30.92%
Val F2 score:  40.40%
Precision:  22.22%
Recall:  50.79%
